<a href="https://colab.research.google.com/github/TheBlackRus/liveproject_HumanPoseEstimation/blob/master/liveproject_humanpose_04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Download the COCO dataset.

In [0]:
# This code downloads the coco dataset from Amazon S3 in parallel.
import boto3
from botocore import UNSIGNED
from botocore.client import Config
import multiprocessing
import subprocess
#files = [ 'train2017.zip']#,'val2017.zip', 'annotations_trainval2017.zip']
files = [ 'val2017.zip', 'annotations_trainval2017.zip']

s3 = boto3.client('s3', config=Config(signature_version=UNSIGNED))
def download_and_unzip_from_s3(file_name, bucket_name='fast-ai-coco'):
    print("Downloading", file_name)
    s3.download_file(bucket_name, file_name, file_name)
    print("Finished downloading", file_name, ". Starting to unzip.")
    subprocess.run(["unzip", file_name])
    print("Finished unzipping", file_name)

# Download in parallel
num_cpus = multiprocessing.cpu_count()
with multiprocessing.Pool(num_cpus) as p:
    p.map(download_and_unzip_from_s3, files)

print("Done transferring all datasets")

Finished downloading annotations_trainval2017.zip . Starting to unzip.
Finished downloading val2017.zip . Starting to unzip.
Finished unzipping annotations_trainval2017.zip
Finished unzipping val2017.zip
Done transferring all datasets


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Generate the input images. 

In [0]:
# read the meta data of the validation set
import json

import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import math
from scipy.io import loadmat
from matplotlib.pyplot import imshow

In [0]:
import json
from torch.utils.data import Dataset
import numpy as np
%matplotlib inline
#from matplotlib.pyplot import imshow
from matplotlib import image
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import cv2
from scipy.ndimage import gaussian_filter

class HumanPoseDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, meta_path,train= False, transform=None):
        
        self.meta = self.load_meta(meta_path)
        self.train = train
        
        self.img_annotations = self.filter_annotations(self.meta["annotations"])
        #self.img_info = meta["images"]
        #self.img_catogory = meta["categories"][0]["keypoints"]
        self.mean = [0.485, 0.456, 0.406]
        self.std = [0.229, 0.224, 0.225]

    def load_meta(self,path):
      
      with open(path, 'r') as json_raw:
          meta = json.load(json_raw)
      return meta

    def filter_annotations(self,meta):
      min_width = 192//4# 48 #192 # 12
      min_height = 256//4#64 # 256 # 16
      print("LEN before filter",len(meta))
      annot = filter(lambda x: x["iscrowd"] == 0 and 
                    (x['bbox'][2]>= min_width and x['bbox'][3]>=min_height) and
                    any(map(lambda y: y>0, x["keypoints"][2:-1:3])) # visible              
                    ,meta)
      annot = list(annot)
      """
      cleaned_annot = []
      for sample in meta:
        no_crowd = sample["iscrowd"] == 0
        start_x,start_y, w,h = sample['bbox']
        not_too_small = w>= min_width and h>=min_height
        inside_box = False
        #print(sample["keypoints"])
        keypoints = sample["keypoints"]
        for i in range(len(keypoints)//3):
          
          keypoint = (keypoints[i*3],keypoints[i*3+1])
          kx,ky = keypoint
          if not (start_x<kx and kx < (start_x+w) and start_y<ky and ky < (start_y+h)):
            inside_box = True
            break
        if all([no_crowd, not_too_small, inside_box]):
          cleaned_annot.append(sample)
        """
      print("AFTER", len(annot))
      return annot #cleaned_annot

    def __len__(self):
        #print(len(self.img_annotations), "anot")
        return len(self.img_annotations)

    def get_image_name(self,annot):
      
      img_file = str(annot["image_id"])
      img_file = img_file.zfill(12) +".jpg" #img_info["file_name"]
      if self.train:
        return "train2017//"+img_file
      else:   
        return "val2017//"+img_file

    def resize_image(self,img, target_width=192, target_height=256):
      img_resized = cv2.resize(img,(target_width,target_height))   
      return img_resized

    def crop_image(self,img, upper_left_corner, size):
      #print(upper_left_corner,size, img.shape)
      start_x, start_y = upper_left_corner
      w,h = size
      #print("CROP",int(start_y),(int(start_y+h-1)),int(start_x),int(start_x+w-1), img.shape)
      img_cropped = img[int(start_y):(int(start_y+h)),int(start_x):int(start_x+w),:]
      #print("CROP", img.shape)
      return img_cropped

    def adjust_keypoint(self,keypoint, start, scale):
      keypoint_x, keypoint_y= keypoint
      start_x,start_y = start
      sx,sy = scale
      x = (keypoint_x-start_x)*sx
      y = (keypoint_y-start_y)*sy
      return x,y

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        annot = self.img_annotations[idx]
        keypoints = annot["keypoints"]

        img_name = self.get_image_name(annot)
        image = io.imread(img_name)
        #
        #print("IDX",idx)
        if len(image.shape) <3:
          # thats a freakin gray image
          image = np.dstack((image, image,image))
        #plt.imshow(image)
        target_width = 192
        target_height = 256

        #print(annot["bbox"],"box")
        start_x,start_y, w,h = annot["bbox"]
        #print("box",annot["bbox"])
        #print(image.shape,"ishape")
        img_crop = self.crop_image(image, (start_x,start_y),(w,h))
        #print("crop",img_crop.shape)
        #plt.imshow(img_crop)
        img_crop = self.resize_image(img_crop,target_width,target_height)
        
        img = img_crop / 255.0
        img = (img-self.mean)/self.std
        # imgshape (256, 192, 3)
        #print("img",img.shape)
        img = np.transpose(img,[2,0,1]) # channel first

        #img_batch = torch.Tensor([img, img])
        #print(img_batch.shape)
        #img_batch = img_batch.permute([0,3,2,1])

        sx = target_width/w 
        sy = target_height/h
        #print("sxsy",sx,sy)

        validity = annot["keypoints"][2::3]

        validity = np.array(validity)
        validity = validity > 0
        validity = validity.astype(np.int)
        #print(validity.shape)

        heatmaps = np.zeros((17,64,48))

       
        for i in range(len(keypoints)//3):
          
          keypoint = (keypoints[i*3],keypoints[i*3+1])
          visible = keypoints[i*3+2]
          #print(keypoint)
          
          x,y = self.adjust_keypoint(keypoint,(start_x,start_y),(sx,sy))
          #print("XY", x,y,int(y//4),int(x//4),sx,sy )
          #not (start_x<kx and kx < (start_x+w) and start_y<ky and ky < (start_y+h))
          kx , ky = keypoint
          #if kx< start_x or ky < start_y or kx> (start_x+w) or ky > (start_y +h):
          if not (start_x<kx and kx < (start_x+w) and start_y<ky and ky < (start_y+h)):
            validity[i] = 0
            #print("FAIL", start_x, kx, start_x+w, start_y,y, start_y+h)
          else:
            
            #if (y//4 >= 64 or x//4 >=47):
            #  print("fail",idx,x,y)
            heatmaps[i,int(y//4),int(x//4)] = 1
            heatmaps[i,:,:] =  gaussian_filter(heatmaps[i,:,:], sigma=2)
            #print(heatmaps[i,:,:].min(),heatmaps[i,:,:].max())
            heatmaps[i,:,:] -= heatmaps[i,:,:].min() 
            heatmaps[i,:,:] /= heatmaps[i,:,:].max()
          #print("N",heatmaps[i,:,:].min(),heatmaps[i,:,:].max())
          #plt.matshow(heatmaps[i,:,:])
          #print(meta["categories"][0]["keypoints"][i])
          
          
        """
        landmarks = self.landmarks_frame.iloc[idx, 1:]
        landmarks = np.array([landmarks])
        landmarks = landmarks.astype('float').reshape(-1, 2)
        sample = {'image': image, 'landmarks': landmarks}

        if self.transform:
            sample = self.transform(sample)
        """
        return torch.Tensor(img), torch.Tensor(heatmaps), torch.Tensor(validity)

In [0]:
data_val = HumanPoseDataset(meta_path="annotations//person_keypoints_val2017.json")
data_train = HumanPoseDataset(meta_path="annotations//person_keypoints_train2017.json", train=True)
#data_train = HumanPoseDataset(meta_path="annotations//person_keypoints_val2017.json", train=False)

LEN before filter 11004
AFTER 4795
LEN before filter 262465
AFTER 112208


In [0]:
for i in range(len(data_val)):
  print(i)
  _, labels, _ = data_val[i]
  
  # h in range(labels.shape[0]):
  #  plt.matshow(labels[h,:,:])

In [0]:
data_val[1]

In [0]:
import torch.nn as nn
import torch.nn.functional as F

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        #block
        self.conv_1 = nn.Conv2d(in_channels = 3, out_channels=64,kernel_size=7,stride=2,padding=3)
        #nn.init.normal_(self.conv_1.weight, std=0.001)
        #nn.init.constant_(self.conv_1.bias,0)
        self.batch_norm_1 = nn.BatchNorm2d(64) # same as out_channels before layer 
        self.relu_1 = nn.ReLU()
        self.pool_1 = nn.MaxPool2d(2, 2)


        self.conv_2 = nn.Conv2d(in_channels = 64, out_channels=128,kernel_size=5,stride=1,padding=2)
        #nn.init.normal_(self.conv_2.weight, std=0.001)
        #nn.init.constant_(self.conv_2.bias,0)
        self.batch_norm_2 = nn.BatchNorm2d(128) # same as out_channels before layer 
        self.relu_2 = nn.ReLU()
        self.pool_2 = nn.MaxPool2d(2, 2)

        self.conv_3 = nn.Conv2d(in_channels= 128, out_channels=256,kernel_size=5,stride=1,padding=2)
        #nn.init.normal_(self.conv_3.weight, std=0.001)
        #nn.init.constant_(self.conv_3.bias,0)
        self.batch_norm_3 = nn.BatchNorm2d(256) # same as out_channels before layer 
        self.relu_3 = nn.ReLU()
        self.pool_3 = nn.MaxPool2d(2, 2)

        # lazy explanition in the workflow...
        # right before
        self.up1 = nn.ConvTranspose2d(256, 256, 4, stride=2,padding=1)
        nn.init.normal_(self.up1.weight, std=0.001)
        self.batch_norm_4 = nn.BatchNorm2d(256) # same as out_channels before layer 
        self.relu_4 = nn.ReLU()

        self.up2 = nn.ConvTranspose2d(256, 256, 4, stride=2,padding=1)
        nn.init.normal_(self.up2.weight, std=0.001)
        self.batch_norm_5 = nn.BatchNorm2d(256) # same as out_channels before layer 
        self.relu_5 = nn.ReLU()

        self.fc1 = nn.Conv2d(in_channels= 256, out_channels=17,kernel_size=1,stride=1,padding=0) #nn.Linear(24 * 4 * 4, 10)# 24 chans x 32//(2*2*2)
        self.sig = nn.Sigmoid()
        
        #nn.init.normal_(self.fc1.weight, std=0.001)
        #nn.init.constant_(self.fc1.bias,0)

        

    def forward(self, x):
        x = self.conv_1(x)
        x = self.batch_norm_1(x)
        x = self.relu_1(x)
        x = self.pool_1(x)

        x = self.conv_2(x)
        x = self.batch_norm_2(x)
        x = self.relu_2(x)
        x = self.pool_2(x)

        x = self.conv_3(x)
        x = self.batch_norm_3(x)
        x = self.relu_3(x)
        x = self.pool_3(x)

        x = self.up1(x)
        x = self.batch_norm_4(x)
        x = self.relu_4(x)

        x = self.up2(x)
        x = self.batch_norm_5(x)
        x = self.relu_5(x)
        



        x = self.fc1(x)#x.view(-1, 24 * 4 * 4)
        x = self.sig(x)
        
        #x = F.relu(self.fc1(x))
        #x = F.relu(self.fc2(x))
        #x = self.fc1(x)
        return x



net = Net()
print(net)

Net(
  (conv_1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
  (batch_norm_1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu_1): ReLU()
  (pool_1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv_2): Conv2d(64, 128, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (batch_norm_2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu_2): ReLU()
  (pool_2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv_3): Conv2d(128, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (batch_norm_3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu_3): ReLU()
  (pool_3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (up1): ConvTranspose2d(256, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (batch_norm_4): BatchNorm2d(256, eps=1e-05, momentum=0.1,

In [0]:
#from torchsummary import summary
#summary(net, (3, 256, 192))

In [0]:
img, heatmaps, validity = data_val[0]

In [0]:
def my_loss(output, target,validity):
    #print("out",output.shape)
    #print("tar",target.shape)
    #print("val",validity.shape)
    """
    for b in range(output.shape[0]):
      for i in range(len(validity[b])):
        torch.
        output[b,i,:,:] = output[b,i,:,:] *validity[b,i]
        target[b,i,:,:] = target[b,i,:,:] *validity[b,i]

    loss = torch.mean((output - target)**2)
    """
    #print("-"*70)
    #print((output-target).shape)
    #print("x"*50)
    #print(validity.shape)
     
    loss = torch.mean(((output - target))**2,(2,3))
    #print("los",loss.shape)
    #print("val",(loss*validity).shape)
    loss = torch.mean( torch.mul(  loss,validity))
    
    
    return loss

In [0]:
def get_max_preds(batch_heatmaps):
    '''
    get predictions from score maps
    heatmaps: numpy.ndarray([batch_size, num_joints, height, width])
    '''
    #assert isinstance(batch_heatmaps, np.ndarray), \
    #    'batch_heatmaps should be numpy.ndarray'
    assert batch_heatmaps.ndim == 4, 'batch_images should be 4-ndim'

    batch_size = batch_heatmaps.shape[0]
    num_joints = batch_heatmaps.shape[1]
    width = batch_heatmaps.shape[3]
    heatmaps_reshaped = batch_heatmaps.reshape((batch_size, num_joints, -1))
    idx = np.argmax(heatmaps_reshaped, 2)
    maxvals = np.amax(heatmaps_reshaped, 2)

    maxvals = maxvals.reshape((batch_size, num_joints, 1))
    idx = idx.reshape((batch_size, num_joints, 1))

    preds = np.tile(idx, (1, 1, 2)).astype(np.float32)

    preds[:, :, 0] = (preds[:, :, 0]) % width
    preds[:, :, 1] = np.floor((preds[:, :, 1]) / width)

    pred_mask = np.tile(np.greater(maxvals, 0.0), (1, 1, 2))
    pred_mask = pred_mask.astype(np.float32)

    preds *= pred_mask
    return preds, maxvals


def calc_dists(preds, target, normalize):
    preds = preds.astype(np.float32)
    target = target.astype(np.float32)
    dists = np.zeros((preds.shape[1], preds.shape[0]))
    for n in range(preds.shape[0]):
        for c in range(preds.shape[1]):
            if target[n, c, 0] > 1 and target[n, c, 1] > 1:
                normed_preds = preds[n, c, :] / normalize[n]
                normed_targets = target[n, c, :] / normalize[n]
                dists[c, n] = np.linalg.norm(normed_preds - normed_targets)
            else:
                dists[c, n] = -1
    return dists


def dist_acc(dists, thr=0.5):
    ''' Return percentage below threshold while ignoring values with a -1 '''
    dist_cal = np.not_equal(dists, -1)
    num_dist_cal = dist_cal.sum()
    if num_dist_cal > 0:
        return np.less(dists[dist_cal], thr).sum() * 1.0 / num_dist_cal
    else:
        return -1

def calc_accuracy(output, target, hm_type='gaussian', thr=0.5):
    '''
    Calculate accuracy according to PCK,
    but uses ground truth heatmap rather than x,y locations
    First value to be returned is average accuracy across 'idxs',
    followed by individual accuracies
    '''
    idx = list(range(output.shape[1]))
    norm = 1.0
    if hm_type == 'gaussian':
        pred, _ = get_max_preds(output)
        target, _ = get_max_preds(target)
        h = output.shape[2]
        w = output.shape[3]
        norm = np.ones((pred.shape[0], 2)) * np.array([h, w]) / 10
    dists = calc_dists(pred, target, norm)

    acc = np.zeros((len(idx) + 1))
    avg_acc = 0
    cnt = 0

    for i in range(len(idx)):
        acc[i + 1] = dist_acc(dists[idx[i]])
        if acc[i + 1] >= 0:
            avg_acc = avg_acc + acc[i + 1]
            cnt += 1

    avg_acc = avg_acc / cnt if cnt != 0 else 0
    if cnt != 0:
        acc[0] = avg_acc
    return avg_acc
def get_accuracy(model, dataloader):

    model.eval()
    # TODO: calculate accuracy
    acc = 0
    count = 0
    #for i, data in enumerate(dataloader, 0):
    i =0
    data = next(dataloader)
    inputs, labels,validity = data
    #labels = labels.squeeze()
    inputs = inputs.cuda()
    
    
    outputs = net(inputs)
    
    #print(predicted, labels)
    o = outputs.cpu().detach().numpy()[:,:,:,:]
    l = labels.detach().numpy()[:,:,:,:]
    #plt.matshow(o[0,0,:,:])
    #print(l.type)
    a = calc_accuracy(o,l)
    #print("a",a)
    acc +=a
    count = count + 1
    #print("AC",acc,count)
    #acc = acc.to(torch.float)
    
    if count >0:
      accuracy = acc/count
    else:
      accuracy = -1
    #outputs = net(inputs)
    model.train()
    
    return accuracy

In [0]:
import torch.optim as optim
from torch.optim.lr_scheduler import MultiStepLR

criterion = my_loss
optimizer = optim.Adam(net.parameters(),lr=0.0001) # ,lr=0.001 #optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

scheduler = MultiStepLR(optimizer, milestones=[5,100], gamma=0.1)

trainloader = torch.utils.data.DataLoader(data_train, batch_size=32,
                                          shuffle=True, num_workers=2)

testloader = torch.utils.data.DataLoader(data_val, batch_size=32,
                                          shuffle=True, num_workers=1)

In [0]:
print("device","cuda" if torch.cuda.is_available() else "cpu")
running_loss = 0.0
eval_iter = iter(testloader)
net.cuda()
print("start training")
for epoch in range(10):  # loop over the dataset multiple times
    

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        #print("-"*60)
        #print("datalen",len(data))
        inputs, labels, validity = data
        #print(labels.shape)
        #for b in range(labels.shape[0]):
        #  for h in range(labels.shape[1]):
        #    plt.matshow(labels[b,h,:,:])
        #print("L",labels.shape)
        #labels = labels.squeeze()
        #print("l",labels.shape)
        inputs = inputs.cuda()
        labels = labels.cuda()
        validity = validity.cuda()
        
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        outputs.cuda()
        #print(labels,labels.shape)
        loss = criterion(outputs, labels,validity)
        loss.backward()
        optimizer.step()
        

        # print statistics
        running_loss += loss.item()
        #print("#"*60)
        #if i % 100 == 10:    # print every 100 mini-batches
    scheduler.step()
    #print("EPOCH",epoch)
    acc = get_accuracy(net,eval_iter)
    print('EPOCH [%3d, %5d] | loss: %.7f | acc %.14f' %(epoch, i + 1, running_loss, acc ))
    

print('Finished Training')

device cuda
start training
EPOCH [  0,  3507] | loss: 19.2072622 | acc 0.30195623140897
EPOCH [  1,  3507] | loss: 7.0061589 | acc 0.39658186200602
EPOCH [  2,  3507] | loss: 6.6062977 | acc 0.37255693824579
EPOCH [  3,  3507] | loss: 6.3698768 | acc 0.46632402356615
EPOCH [  4,  3507] | loss: 6.2014352 | acc 0.43743019499742
EPOCH [  5,  3507] | loss: 6.0711500 | acc 0.49007533220336
EPOCH [  6,  3507] | loss: 5.9642977 | acc 0.45647043195201
EPOCH [  7,  3507] | loss: 5.8757120 | acc 0.47581655772272
EPOCH [  8,  3507] | loss: 5.7967115 | acc 0.46660789904511


KeyboardInterrupt: ignored

4.2

In [0]:
import torch
import torch.nn as nn
#from torch.utils import load_state_dict_from_url


__all__ = ['ResNet', 'resnet18', 'resnet34', 'resnet50', 'resnet101',
           'resnet152', 'resnext50_32x4d', 'resnext101_32x8d',
           'wide_resnet50_2', 'wide_resnet101_2']


model_urls = {
    'resnet18': 'https://download.pytorch.org/models/resnet18-5c106cde.pth',
    'resnet34': 'https://download.pytorch.org/models/resnet34-333f7ec4.pth',
    'resnet50': 'https://download.pytorch.org/models/resnet50-19c8e357.pth',
    'resnet101': 'https://download.pytorch.org/models/resnet101-5d3b4d8f.pth',
    'resnet152': 'https://download.pytorch.org/models/resnet152-b121ed2d.pth',
    'resnext50_32x4d': 'https://download.pytorch.org/models/resnext50_32x4d-7cdf4587.pth',
    'resnext101_32x8d': 'https://download.pytorch.org/models/resnext101_32x8d-8ba56ff5.pth',
    'wide_resnet50_2': 'https://download.pytorch.org/models/wide_resnet50_2-95faca4d.pth',
    'wide_resnet101_2': 'https://download.pytorch.org/models/wide_resnet101_2-32ee1156.pth',
}


def conv3x3(in_planes, out_planes, stride=1, groups=1, dilation=1):
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=dilation, groups=groups, bias=False, dilation=dilation)


def conv1x1(in_planes, out_planes, stride=1):
    """1x1 convolution"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None, groups=1,
                 base_width=64, dilation=1, norm_layer=None):
        super(BasicBlock, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        if groups != 1 or base_width != 64:
            raise ValueError('BasicBlock only supports groups=1 and base_width=64')
        if dilation > 1:
            raise NotImplementedError("Dilation > 1 not supported in BasicBlock")
        # Both self.conv1 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = norm_layer(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = norm_layer(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out


class Bottleneck(nn.Module):
    # Bottleneck in torchvision places the stride for downsampling at 3x3 convolution(self.conv2)
    # while original implementation places the stride at the first 1x1 convolution(self.conv1)
    # according to "Deep residual learning for image recognition"https://arxiv.org/abs/1512.03385.
    # This variant is also known as ResNet V1.5 and improves accuracy according to
    # https://ngc.nvidia.com/catalog/model-scripts/nvidia:resnet_50_v1_5_for_pytorch.

    expansion = 4

    def __init__(self, inplanes, planes, stride=1, downsample=None, groups=1,
                 base_width=64, dilation=1, norm_layer=None):
        super(Bottleneck, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        width = int(planes * (base_width / 64.)) * groups
        # Both self.conv2 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv1x1(inplanes, width)
        self.bn1 = norm_layer(width)
        self.conv2 = conv3x3(width, width, stride, groups, dilation)
        self.bn2 = norm_layer(width)
        self.conv3 = conv1x1(width, planes * self.expansion)
        self.bn3 = norm_layer(planes * self.expansion)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out


class ResNet(nn.Module):

    def __init__(self, block, layers, num_classes=1000, zero_init_residual=False,
                 groups=1, width_per_group=64, replace_stride_with_dilation=None,
                 norm_layer=None):
        super(ResNet, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        self._norm_layer = norm_layer

        self.inplanes = 64
        self.dilation = 1
        if replace_stride_with_dilation is None:
            # each element in the tuple indicates if we should replace
            # the 2x2 stride with a dilated convolution instead
            replace_stride_with_dilation = [False, False, False]
        if len(replace_stride_with_dilation) != 3:
            raise ValueError("replace_stride_with_dilation should be None "
                             "or a 3-element tuple, got {}".format(replace_stride_with_dilation))
        self.groups = groups
        self.base_width = width_per_group
        self.conv1 = nn.Conv2d(3, self.inplanes, kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.bn1 = norm_layer(self.inplanes)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2,
                                       dilate=replace_stride_with_dilation[0])
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2,
                                       dilate=replace_stride_with_dilation[1])
        
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2,
                                       dilate=replace_stride_with_dilation[2])
        
        #self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        #self.fc = nn.Linear(512 * block.expansion, num_classes)

        self.up1 = nn.ConvTranspose2d(2048, 256, 4, stride=2,padding=1)
        nn.init.normal_(self.up1.weight, std=0.001)
        self.batch_norm_4 = nn.BatchNorm2d(256) # same as out_channels before layer 
        self.relu_4 = nn.ReLU()

        self.up2 = nn.ConvTranspose2d(256, 256, 4, stride=2,padding=1)
        nn.init.normal_(self.up2.weight, std=0.001)
        self.batch_norm_5 = nn.BatchNorm2d(256) # same as out_channels before layer 
        self.relu_5 = nn.ReLU()

        self.up3 = nn.ConvTranspose2d(256, 256, 4, stride=2,padding=1)
        nn.init.normal_(self.up3.weight, std=0.001)
        self.batch_norm_6 = nn.BatchNorm2d(256) # same as out_channels before layer 
        self.relu_6 = nn.ReLU()

        self.fc1 = nn.Conv2d(in_channels= 256, out_channels=17,kernel_size=1,stride=1,padding=0) #nn.Linear(24 * 4 * 4, 10)# 24 chans x 32//(2*2*2)
        self.sig = nn.Sigmoid()


        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

        # Zero-initialize the last BN in each residual branch,
        # so that the residual branch starts with zeros, and each residual block behaves like an identity.
        # This improves the model by 0.2~0.3% according to https://arxiv.org/abs/1706.02677
        if zero_init_residual:
            for m in self.modules():
                if isinstance(m, Bottleneck):
                    nn.init.constant_(m.bn3.weight, 0)
                elif isinstance(m, BasicBlock):
                    nn.init.constant_(m.bn2.weight, 0)

    def _make_layer(self, block, planes, blocks, stride=1, dilate=False):
        norm_layer = self._norm_layer
        downsample = None
        previous_dilation = self.dilation
        if dilate:
            self.dilation *= stride
            stride = 1
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes * block.expansion, stride),
                norm_layer(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample, self.groups,
                            self.base_width, previous_dilation, norm_layer))
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes, groups=self.groups,
                                base_width=self.base_width, dilation=self.dilation,
                                norm_layer=norm_layer))

        return nn.Sequential(*layers)

    def _forward_impl(self, x):
        # See note [TorchScript super()]
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        
        x = self.up1(x)
        x = self.batch_norm_4(x)
        x = self.relu_4(x)

        x = self.up2(x)
        x = self.batch_norm_5(x)
        x = self.relu_5(x)

        x = self.up3(x)
        x = self.batch_norm_6(x)
        x = self.relu_6(x)
        
        x = self.fc1(x)#x.view(-1, 24 * 4 * 4)
        x = self.sig(x)
        
        

        return x

    def forward(self, x):
        return self._forward_impl(x)


def _resnet(arch, block, layers, pretrained, progress, **kwargs):
    model = ResNet(block, layers, **kwargs)
    if pretrained:
        state_dict = load_state_dict_from_url(model_urls[arch],
                                              progress=progress)
        model.load_state_dict(state_dict)
    return model


def resnet18(pretrained=False, progress=True, **kwargs):
    r"""ResNet-18 model from
    `"Deep Residual Learning for Image Recognition" <https://arxiv.org/pdf/1512.03385.pdf>`_

    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    return _resnet('resnet18', BasicBlock, [2, 2, 2, 2], pretrained, progress,
                   **kwargs)



def resnet34(pretrained=False, progress=True, **kwargs):
    r"""ResNet-34 model from
    `"Deep Residual Learning for Image Recognition" <https://arxiv.org/pdf/1512.03385.pdf>`_

    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    return _resnet('resnet34', BasicBlock, [3, 4, 6, 3], pretrained, progress,
                   **kwargs)



def resnet50(pretrained=False, progress=True, **kwargs):
    r"""ResNet-50 model from
    `"Deep Residual Learning for Image Recognition" <https://arxiv.org/pdf/1512.03385.pdf>`_

    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    return _resnet('resnet50', Bottleneck, [3, 4, 6, 3], pretrained, progress,
                   **kwargs)



def resnet101(pretrained=False, progress=True, **kwargs):
    r"""ResNet-101 model from
    `"Deep Residual Learning for Image Recognition" <https://arxiv.org/pdf/1512.03385.pdf>`_

    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    return _resnet('resnet101', Bottleneck, [3, 4, 23, 3], pretrained, progress,
                   **kwargs)



def resnet152(pretrained=False, progress=True, **kwargs):
    r"""ResNet-152 model from
    `"Deep Residual Learning for Image Recognition" <https://arxiv.org/pdf/1512.03385.pdf>`_

    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    return _resnet('resnet152', Bottleneck, [3, 8, 36, 3], pretrained, progress,
                   **kwargs)



def resnext50_32x4d(pretrained=False, progress=True, **kwargs):
    r"""ResNeXt-50 32x4d model from
    `"Aggregated Residual Transformation for Deep Neural Networks" <https://arxiv.org/pdf/1611.05431.pdf>`_

    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    kwargs['groups'] = 32
    kwargs['width_per_group'] = 4
    return _resnet('resnext50_32x4d', Bottleneck, [3, 4, 6, 3],
                   pretrained, progress, **kwargs)



def resnext101_32x8d(pretrained=False, progress=True, **kwargs):
    r"""ResNeXt-101 32x8d model from
    `"Aggregated Residual Transformation for Deep Neural Networks" <https://arxiv.org/pdf/1611.05431.pdf>`_

    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    kwargs['groups'] = 32
    kwargs['width_per_group'] = 8
    return _resnet('resnext101_32x8d', Bottleneck, [3, 4, 23, 3],
                   pretrained, progress, **kwargs)



def wide_resnet50_2(pretrained=False, progress=True, **kwargs):
    r"""Wide ResNet-50-2 model from
    `"Wide Residual Networks" <https://arxiv.org/pdf/1605.07146.pdf>`_

    The model is the same as ResNet except for the bottleneck number of channels
    which is twice larger in every block. The number of channels in outer 1x1
    convolutions is the same, e.g. last block in ResNet-50 has 2048-512-2048
    channels, and in Wide ResNet-50-2 has 2048-1024-2048.

    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    kwargs['width_per_group'] = 64 * 2
    return _resnet('wide_resnet50_2', Bottleneck, [3, 4, 6, 3],
                   pretrained, progress, **kwargs)



def wide_resnet101_2(pretrained=False, progress=True, **kwargs):
    r"""Wide ResNet-101-2 model from
    `"Wide Residual Networks" <https://arxiv.org/pdf/1605.07146.pdf>`_

    The model is the same as ResNet except for the bottleneck number of channels
    which is twice larger in every block. The number of channels in outer 1x1
    convolutions is the same, e.g. last block in ResNet-50 has 2048-512-2048
    channels, and in Wide ResNet-50-2 has 2048-1024-2048.

    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    kwargs['width_per_group'] = 64 * 2
    return _resnet('wide_resnet101_2', Bottleneck, [3, 4, 23, 3],
                   pretrained, progress, **kwargs)

In [0]:
from torch.utils.model_zoo import load_url
model = resnet50()
state_dict = load_url(model_urls['resnet50'],progress=True)

model.load_state_dict(state_dict,strict=False)

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/checkpoints/resnet50-19c8e357.pth


_IncompatibleKeys(missing_keys=['up1.weight', 'up1.bias', 'batch_norm_4.weight', 'batch_norm_4.bias', 'batch_norm_4.running_mean', 'batch_norm_4.running_var', 'up2.weight', 'up2.bias', 'batch_norm_5.weight', 'batch_norm_5.bias', 'batch_norm_5.running_mean', 'batch_norm_5.running_var', 'up3.weight', 'up3.bias', 'batch_norm_6.weight', 'batch_norm_6.bias', 'batch_norm_6.running_mean', 'batch_norm_6.running_var', 'fc1.weight', 'fc1.bias'], unexpected_keys=['fc.weight', 'fc.bias'])

Freeze layers

In [0]:
model.layer1.requires_grad_ = False
model.layer2.requires_grad_ = False
model.layer3.requires_grad_ = False
model.layer4.requires_grad_ = False

In [0]:
net = model

In [0]:
from torchsummary import summary
net.cuda()
summary(net, (3, 256, 192))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 64, 128, 96]           9,408
       BatchNorm2d-2          [-1, 64, 128, 96]             128
              ReLU-3          [-1, 64, 128, 96]               0
         MaxPool2d-4           [-1, 64, 64, 48]               0
            Conv2d-5           [-1, 64, 64, 48]           4,096
       BatchNorm2d-6           [-1, 64, 64, 48]             128
              ReLU-7           [-1, 64, 64, 48]               0
            Conv2d-8           [-1, 64, 64, 48]          36,864
       BatchNorm2d-9           [-1, 64, 64, 48]             128
             ReLU-10           [-1, 64, 64, 48]               0
           Conv2d-11          [-1, 256, 64, 48]          16,384
      BatchNorm2d-12          [-1, 256, 64, 48]             512
           Conv2d-13          [-1, 256, 64, 48]          16,384
      BatchNorm2d-14          [-1, 256,

In [0]:
import torch.optim as optim
from torch.optim.lr_scheduler import MultiStepLR

criterion = my_loss
optimizer = optim.Adam(net.parameters(),lr=0.0001) # ,lr=0.001 #optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

scheduler = MultiStepLR(optimizer, milestones=[10,100], gamma=0.1)

trainloader = torch.utils.data.DataLoader(data_train, batch_size=32,
                                          shuffle=True, num_workers=2)

testloader = torch.utils.data.DataLoader(data_val, batch_size=32,
                                          shuffle=True, num_workers=1)

In [0]:
print("device","cuda" if torch.cuda.is_available() else "cpu")
running_loss = 0.0
eval_iter = iter(testloader)
net.cuda()
print("start training")
for epoch in range(10):  # loop over the dataset multiple times
    

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        #print("-"*60)
        #print("datalen",len(data))
        inputs, labels, validity = data
        #print(labels.shape)
        #for b in range(labels.shape[0]):
        #  for h in range(labels.shape[1]):
        #    plt.matshow(labels[b,h,:,:])
        #print("L",labels.shape)
        #labels = labels.squeeze()
        #print("l",labels.shape)
        inputs = inputs.cuda()
        labels = labels.cuda()
        validity = validity.cuda()
        
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        outputs.cuda()
        #print(labels,labels.shape)
        loss = criterion(outputs, labels,validity)
        loss.backward()
        optimizer.step()
        

        # print statistics
        running_loss += loss.item()
        #print("#"*60)
        #if i % 100 == 10:    # print every 100 mini-batches
    scheduler.step()
    #print("EPOCH",epoch)
    acc = get_accuracy(net,eval_iter)
    print('EPOCH [%3d, %5d] | loss: %.7f | acc %.14f' %(epoch, i + 1, running_loss, acc ))
    

print('Finished Training')

device cuda
start training
EPOCH [  0,  3507] | loss: 15.2324904 | acc 0.56702353154453
EPOCH [  1,  3507] | loss: 4.7137077 | acc 0.66735544297464
EPOCH [  2,  3507] | loss: 4.0416069 | acc 0.71747531031078
EPOCH [  3,  3507] | loss: 3.6553278 | acc 0.73937850366937
EPOCH [  4,  3507] | loss: 3.3486448 | acc 0.74424188391496
EPOCH [  5,  3507] | loss: 3.0745666 | acc 0.75546536815580
EPOCH [  6,  3507] | loss: 2.8259762 | acc 0.68888745701394
EPOCH [  7,  3507] | loss: 2.5946723 | acc 0.73543317333868
EPOCH [  8,  3507] | loss: 2.3773787 | acc 0.69094696007035
EPOCH [  9,  3507] | loss: 2.1824468 | acc 0.68622314344633
Finished Training
